# Result analysis

### Reading files in

In [128]:
import duckdb

%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb
%config SqlMagic.displaylimit = None

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


displaylimit: Value None will be treated as 0 (no limit)

In [219]:
%%sql
drop table if exists article_text;
CREATE TABLE article_text AS
    SELECT *
    FROM read_json('../../src/datasets/silver/scanned/full_scope_data/all_articles.json');

drop table if exists rubrick;
CREATE TABLE rubrick AS
    SELECT *
    FROM read_json('../../src/datasets/silver/scanned/full_scope_data/mod_rubrick.jsonl');

drop table if exists ner;
CREATE TABLE ner AS
    SELECT *
    FROM read_json('../../src/datasets/silver/scanned/full_scope_data/NER.jsonl');

drop table if exists accuracy;
CREATE TABLE accuracy AS
    SELECT *
    FROM read_csv('../transkribus/transkribus.csv',header=True,sep=',');


drop table if exists results;
CREATE TABLE results AS
    SELECT *
    FROM read_json('../../src/datasets/silver/scanned/full_scope_data/responses.jsonl');


drop table if exists manual;
CREATE TABLE manual AS
    SELECT *
    FROM read_json('../../src/datasets/silver/scanned/full_scope_data/text_and_label.jsonl');


Running query in 'duckdb'

Count
4487


### Checking confidence levels for ESTBERT

In [130]:
%%sql

select CAST(confidence * 10 AS INT), count(*) FROM rubrick rbr
INNER JOIN article_text at
ON rbr.id = at.article_id
group by 1
--WHERE rbr.confidence < 0.7
--LIMIT 10;

Running query in 'duckdb'

CAST((confidence * 10) AS INTEGER),count_star()
8,71
6,63
4,6
7,66
9,167
5,47
10,4298


In [131]:
%%sql
select rbr.id, rbr.label, at.modernized_text FROM rubrick rbr
INNER JOIN article_text at
ON rbr.id = at.article_id
LEFT JOIN ner n
ON rbr.id = n.article_id
WHERE len(at.modernized_text) > 100
limit 1


Running query in 'duckdb'

id,label,modernized_text
eestikirikeelk19260225.1.6_P6_TB00002,ARVAMUS,"kes teda siiamaani põlgasid. Tema ei oleks mitte väiksem, kui teda kõik inimesed valesti hindaksid. Jeesus on. Jeesus oli. Jeesus tuleb. Jeesus on mõõt kõigile inimestele ja kõigil aegul. Aga õnn ja õndsus on neil, kes tema juure tulevad. 3a mõnele, kes kõigub, on toeks, kui ka mõningad targad liha poolest tunnustama sõna leiavad selle jaoks, kes kõikide õndsus ja kõikide kohtumõistja on. u. Leben J. Varik."


In [132]:
%%sql
select
count(*) AS Total,
SUM(case when lower(trim(kp)) LIKE '%' || lower(label) || '%' THEN 1  else 0 end) AS Matched,
Total - matched AS Not_Matched,
SUM(case when lower(trim(kp))  NOT LIKE '%' || lower(label) || '%' AND label = 'ARVAMUS' THEN 1 ELSE 0 END) AS Unmatched_Opinion
--SUM(case when lower(trim(kp))  NOT LIKE '%' || lower(label) || '%' AND label = 'KRIMI' THEN 1 ELSE 0 END) AS Not_Matched_Arvamus,
--SUM(case when lower(trim(kp))   LIKE '%' || lower(label) || '%' AND label = 'KRIMI' THEN 1 ELSE 0 END) AS Matched_Krimi
--id, label, kp, case when lower(trim(kp)) LIKE '%' || lower(label) || '%' THEN 1  else 0 end as matched,
--id, label, kp, case when label LIKE '%' || kp || '%' THEN 1  else 0 end as matched,
--lower(label), lower(kp)
FROM manual 

where
1=1
and len(kp) > 0
--and matched = 1
--and label = 'ARVAMUS'

Running query in 'duckdb'

Total,Matched,Not_Matched,Unmatched_Opinion
143,87,56,42


In [211]:
%%sql
SELECT * from RESULTS 
LIMIT 1

Running query in 'duckdb'

qry_id,id,content,avgLogprobs,modelVersion
1,eestikirikeelk19260225.1.6_P6_TB00001,"[1] Religion: Discusses aspects of faith, religious institutions, and religious education.",-0.24096162695633738,gemini-2.0-flash


### Analysis of current results

In [ ]:
%%sql
--COPY(
select acc.Correct_Percent, MAX(SPLIT_PART(res.content, ':', 1)) AS topic_lvl1,  rbr.id, rbr.label, at.modernized_text FROM rubrick rbr
INNER JOIN article_text at
ON rbr.id = at.article_id
LEFT JOIN ner n
ON rbr.id = n.article_id
LEFT JOIN accuracy acc
on acc.newspaper = LEFT(rbr.id, LENGTH(rbr.id) - 11)
LEFT JOIN results res
on res.id = rbr.id

where 1=1
AND len(res.content) > 0
AND res.content not like '%[2]%'
--AND LEFT(rbr.id, LENGTH(rbr.id) - 11) = 'postimeesew19390716.1.1'
--acc.Correct_Percent > 80
--AND modernized_text like '%hoone%'
group by acc.Correct_Percent, rbr.id, rbr.label, at.modernized_text
order by len(at.modernized_text) desc
--) TO '../../src/datasets/silver/scanned/full_scope_data/only_topic_1.jsonl' (FORMAT JSON);
--limit 20


Running query in 'duckdb'

Count
95


### Grouping results by topic lvl 1

In [246]:
%%sql
--COPY (
  select * FROM(
  SELECT 
   -- acc.Correct_Percent, 
    MAX(split_part(res.content, ':', 1)) AS topic_lvl1, 
    rbr.id, 
    m.kp,
    --rbr.label, 
    at.modernized_text
  FROM 
    rubrick rbr
  INNER JOIN 
    article_text at
  ON 
    rbr.id = at.article_id
  LEFT JOIN 
    ner n
  ON 
    rbr.id = n.article_id
  LEFT JOIN 
    accuracy acc
  ON 
    acc.newspaper = LEFT(rbr.id, LENGTH(rbr.id) - 11)
  LEFT JOIN 
    results res
  ON 
    res.id = rbr.id
  LEFT JOIN manual m
on m.id = rbr.id
  WHERE 
  1=1
  AND len(res.content) > 0
  AND trim(res.content) <> 'None'
    --split_part(res.content, ':', 1) = '[1] World news'
  GROUP BY 
    acc.Correct_Percent, 
    rbr.id, 
    rbr.label, 
    at.modernized_text,
    m.kp
) a 
--) TO '../../src/datasets/silver/scanned/full_scope_data/lopp.jsonl' (FORMAT JSON);


Running query in 'duckdb'

topic_lvl1,id,kp,modernized_text
[1] Weather[2] Rainfall (Document,postimeesew19390716.1.1_P1_TB00039,ILM,"Möödunud nädalal olid hoogja äikesevihmad igapäevased nähted. Püsivamat lausvihma ei ole Eestis tänavu kevade ega suve jooksul üldse sadanud. Selles aga seisabki Mr kurbloolus. Hoogvihmad, nagu ikka, on alati paigutise iseloomuga ja jätavad sageli üksikud maakohad niisutamata. Nii on see juhtunud ka käesoleval juulikuul. Saaremaal võidutseb paiguti kevadine põud veel edasi. Kuigi kõigist sademete jaamadest andmed veel puuduvad, võib siiski ütelda, et Eesti kõige läänepoolseni maa Vilsandi ei ole juulikuus ainustki piiska vihma saanud. Juuli algusest kuni 15. juulini on kõige enam sadanud Vaindlo saarel 36 mm. Edasi Paldiskis, Pärnus ja Viljandis 32 mm, Narva-Jõesuus 26 mm ja teistes kohtades vähem. Peab märkima, et Vilsandis ka juuni lõpupoolel väga vähe vihma tuli. airuzlt paaril kokku 7 mm. Möödunud nädala kestel püsis Euroopa enamuses madala õhurõhu seisu juures. Inglismaalt tulev madalrõhkkond siirdus Skandinaaviasse ja tungis kuni Valge mereni, kust ta aeglaselt tagasi liikus ja sealjuures täitus. Nädala lõpul oli ta juba kadunud. Nimetatud madalrõhkkonna mõjul levisid Skandinaavia jahedamad õhumassid Balti riikidesse ja osalt ka Kesk-Eurööpasse, Põhjustades laialdaselt äikest ja hoogvihma. Temperatuur Eestis ei tõusnud kcskpäeval Palju üle 20-ne kraadi ja ööd sageli olid üsna jahedad. Lõuna-Euroopa ja Venemaa Püsisid edasi soojade õhumasside valduses. Eriti Lõunaja KaguVenes valitsevad juba kauemat aega väga"
None,sakalaew19361118.1.6_P6_TB00018,"Eesti, haridus","Tõrva N. M. K. Ü., mis tegutsenud võrdlemisi edukalt juba 7 aastat, on oma senises töös päätähelepanu suuninud koolinoortele, mis seletatav osalt gümnaasiuminoorte enamusega noorjuhtide seas. Käesoleval sügisel asuti aga seisukohale, et enam tähelepanu koondada tuleb just neile poistele, kes algkooli lõpetanud ja asunud mõnele kutsealale õpilaseks. Senised koolinoorte klubid jäävad küll edasi töötama, kuid juba enam vastava kooli kontrolli all."
[1] Helme ruins[2] Preservation (Document,sakalaew19361118.1.6_P6_TB00020,"Eesti,varemed","Helme varemetes on viimastel aastatel tulnud nähtavale selle müstilisema osa kuulsate maaaluste käikude hävimise hädaoht. Käikude kohal mäepeal kasvavate puude juured on läbiuuristanud muidu tugeva liivakivi ja see langeb seintelt lademete kaupa alla. Samal põhjusel on sisse kukkunud aastate jooksul juba 2 „põrguhaua viimase aastakümne sees. Oleks soovitav ettevõtta midagi käikude hävimise vastu kas või üksi sellepärast, et nad tihedalt seotud meie rahvaeepose „Kalevipojaga lugu on terveni siit kirja pandud. Puude juured on mõjunud kahjustavalt ka vana ordulossi varemetele, kus ajajooksul järjest enam muidu tugevaid kivimüüre on mahalangenud, kuna juured on need lahti murdnud."
[1] Estonian local news,jarvateatajaew19340104.1.4_P4_TB00010,"ARVAMUS, KULTUUR, HEATEGEVUS","Järgnes naiskodukaitsjate korraldatud vaeslaste jõulupuu. Virgad naised olid valmistatmd kohalikele tvaeslastele hulga kingitust. Samuti kinkis kohalik ärimees M. isiklikult paljudele. Võõrastavana tundus, et keegi lapsevanem saadud kingituse uhke shestiga tagasi saatis. Selline kasvatamatuscst sõltuv nähe peaks edaspidi aitajate tähelepanu pälvima, et vältida kaastundlikkude südamete piinlikkust. Muidu oli õhtu kõigiti kodune. Samade korraldajate poolt viidi jõülulaupäeval kohalikku vaestemajja kingitusi. Igatahes tänuväärt algatus ja esmakordne siin."
[1] Estonian local news,jarvateatajaew19340104.1.4_P4_TB00052,,"Vaimuiik laulupäev Tapal. Eevlevai suvel korraldab Tapa Jakobi koguduse nõukogu Tapal vaimuliku laulu-päeva. Puhas sissetulek laulupäevast läheb kiriku võlgade katteks. Kutsed saadetakse kõikidele ümbruskonna sa naaberlinnade kooridele. Laulupäeva täpne aeg on veel kindlaks määvamata'. Amblas ei korraldata enam hmviõhtuid. Varematel aastatel korraldati Amblas tvlviti ikka järjekindlalt hum!õhtuid referaatidega, muusikaliste ettekannetega jne., mille

## Check for lvl2 topics

In [227]:
%%sql
select 
T1,
list(distinct TRIM(T2)) FROM  (
select 

SPLIT_PART(SPLIT_PART(SPLIT_PART(res.content, '\n', 1), ':',1),'[2]',1) AS T1,
SPLIT_PART(SPLIT_PART(SPLIT_PART(SPLIT_PART(res.content, '\n', 1), ':',1),'[2]',2),'(',1) AS T2 

from results res
WHERE len(T2) > 0
--limit 5
) AS iq
group by 1
;

Running query in 'duckdb'

T1,"list(DISTINCT main.""trim""(T2))"
[1] Helme ruins,['Preservation']
[1] Estonian local news,"['Education', 'Culture', 'Politics', 'History', 'Crime', 'Social event', 'Infrastructure', 'Land Disputes', 'Student Affairs', 'Business', 'Social Event', 'Economy', 'Newspaper Contact Information', 'Scouting', 'Archaeology', 'Announcements', 'Religious Services']"
[1] Politics,['National Holiday']
[1] Weather,"['Rainfall', 'Agricultural Fair']"
[1] World news,"['Film review', 'Business', 'International Relations', 'Land reform', 'International Trade', 'Military', 'Media', 'Religion', 'Political Affairs', 'Crime', 'Political conflict', 'Education', 'Agriculture and Land Management', 'Health', 'Politics', 'Travel', 'Entertainment', 'Infrastructure']"
[1] Agriculture,"['Animal Husbandry', 'Agricultural Education', 'Agricultural Technology', 'Livestock Competition']"
[1] Law and Crime,['Theft']
[1] Education,['Teacher Organization']
[1] Crime,['Punishment']
"[1] World news[1] World news,",['Political conflict']


## Check result againsta manual labeling

In [228]:
%%sql 
select len(id) FROM results
order by 1 desc
--limit 1

Running query in 'duckdb'

len(id)
38
38
38
38
38
38
38
38
38
38


In [229]:
%%sql
SELECT * FROM manual m
INNER JOIN results r
on m.id = r.id
and r.qry_id = '4'
and len(m.kp) > 0
--limit 1;

Running query in 'duckdb'

id,label,kp,modernized_text,qry_id,id_1,content,avgLogprobs,modelVersion
postimeesew19390716.1.1_P1_TB00004,EESTI,AJALEHT,"Toimetus ja peakontor Tartus, Ülikooli t. 21/23. Toimetus ja kontor Tallinnas, Viru tn. nr. 11. Telefonid Tartus: Kella 9—15 kodukeskjaam kõigiks väljakutseteks 40-80, toimetuse kaugekõned 29-86 raamatukauplus 22-50. Peale kella 15 ja pühapäevadel toimetuse kaugekõned 29-86, toimetus (kohalik osakond, sport, üldised teated) 40-81, ajalehe trükikoda ja toimetuse öövalve peale keskööd 40-82, kontor 40-83, peatoimetaja ja toimetuse sekretär 40-84.",4,postimeesew19390716.1.1_P1_TB00004,"[1] Estonian local news[2] Newspaper Contact Information: Provides contact information for the newspaper's offices in Tartu and Tallinn.Quote: ""Toimetus ja peakontor Tartus, Ülikooli t. 21/23. Toimetus ja kontor Tallinnas, Viru tn. nr. 11.""",-0.0063365745544433595,gemini-2.0-flash
postimeesew19390716.1.1_P1_TB00006,EESTI,AJALEHT,"Kuulutusi võetakse vastu äripäeviti: „Postimehe kontoris kl. 8.30-st kuni 18-ni. „Postimehe ses, Suurturg 16, kl. 8-st kuni 17-ni, „Postimehe kontoris kl. 9-st kuni 15.30-ni. Pühapäeval: „Postimehe peakontoris tel. 40-83, kl. 18 kuni 20-ni.",4,postimeesew19390716.1.1_P1_TB00006,"[1] Estonian local news[2] Newspaper Contact Information: Provides contact information for placing announcements in the ""Postimees"" newspaper.Quote: ""Kuulutusi võetakse vastu äripäeviti: „Postimehe kontoris kl. 8.30-st kuni 18-ni.""",-0.037461985860552105,gemini-2.0-flash
kaja19290813-1.1.7_P7_TB00028,KULTUUR,"uritus, eesti",Näitusmessil tahetakse eritähelepanu juhtida Eesti majanduselule ja saadustele. Eesti saadused varustatakse messil sellekohaste plakatitega. Näitus-messi kinos näidatakse filme,4,kaja19290813-1.1.7_P7_TB00028,"[1] Estonian local news[2] Economy: Mentions focus on Estonian economy. ""Näitusmessil tahetakse eritähelepanu juhtida Eesti majanduselule ja saadustele.""",-0.0982445757439796,gemini-2.0-flash
postimeesew19330401.1.7_P7_TB00006,EESTI,"majandus,eesti","Neil päevil jõudsid Tartu esimesed . palgiparved, mis läksid A. Univeri ja Fur! mansky lauavabrikutesse ümbertöotamij seks. Puuvedu aga pole veel alanud, sest j Pedja ja Kärevere kanal on jääs. Nende l veeteede kallastel asuvad aga kõige suuremad küttepuu tagavarad.",4,postimeesew19330401.1.7_P7_TB00006,"[1] Estonian local news[2] Business: Mentions local businesses (lauavabrikutesse) and their operations related to timber.Quote: ""Neil päevil jõudsid Tartu esimesed . palgiparved, mis läksid A. Univeri ja Fur! mansky lauavabrikutesse ümbertöotamij seks.""[2] Infrastructure: Mentions waterways (Pedja ja Kärevere kanal) used for transportation and the impact of ice on their use.Quote: ""Puuvedu aga pole veel alanud, sest j Pedja ja Kärevere kanal on jääs.""",-0.09674685036958154,gemini-2.0-flash
postimeesew19330401.1.7_P7_TB00012,EESTI,EESTI,"Auvävava maja lammutamisest saadud ehitusmaterjal tarvitatakse Tähtvere alla ehitatava rannapaviljöni Püstitamiseks. Reedel viidi esimene lodja4äjK materjali kohale ja lähematel päevadel viiakse veel teine lodjatäis. Paviljoni ehitamine kohapeal algas kohe. Paviljon tuleb täidetud kaldaäärele, endise luhavahimaja asemele.",4,postimeesew19330401.1.7_P7_TB00012,"[1] Estonian local news[2] Infrastructure: Mentions construction of a beach pavilion and use of building materials. ""Auvävava maja lammutamisest saadud ehitusmaterjal tarvitatakse Tähtvere alla ehitatava rannapaviljöni Püstitamiseks.""",-0.14695213780258642,gemini-2.0-flash
postimeesew19330401.1.7_P7_TB00016,EESTI,"matused,EESTI",Neljapasval maeti Tartu Peetri tvanal kalmistul maa mulda üliõp. Verner Treumam põrm. Matuse talitust toimetas õp. Paul Kuusik Tallinnast. Matusest vSttis osa hulk lahkunu tuttavaid. Matust jälgis suur rahvahull ja kaastvõitlejaid. Samuti suure rahtva hulga osatvõtuil ?naeti Tartu-Maarja kalmistule neljapäeval pärast Lõunat Tallinna tveretöö ohtver Voldemar Maasingi põrm.,4,postimeesew19330401.1.7_P7_TB00016,"[1] Estonian local n